Before you turn this problem set in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Note that in code sections, you must replace `stop("Not Implemented")` with your code. Otherwise, you will have points automatically deducted in the grading process.

**Please do not rename this file.**

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER/EXPLANATION HERE". In addition, please do not include your name on this assignment to ensure anonymity for the peer reviews.

---

# Problem 1

This homework will require the use of a few datasets in a package written by the TA, Dan. Before beginning, please be sure to download and install the package. Example code for doing this is below:

In [ ]:
# Un-comment the line below to install the package by deleting the # symbol
install.packages("C:/Users/Arghyadeep/Downloads/AMSdata_0.1.0.tar.gz",repos = NULL, type = "source") # Change the first character string to the path of the source package 
library(AMSdata)

The `election_2016` dataset contains county-level information about the porportion of votes received in the presidential primaries for each candidate in 2016. Begin by subsetting the data to just the information concerning then-candidate Donald Trump. Name this subsetted data set `trump_2016`.

In [ ]:
data("election_2016")
# YOUR CODE HERE
x = election_2016
trump_2016 = x[which(x$candidate=="Donald Trump"),]
trump_2016$state = NULL
trump_2016$state_abbreviation = NULL
trump_2016$county = NULL
trump_2016$party = NULL
trump_2016$candidate = NULL

Now quickly check to make sure the subset was done correctly before moving on.

In [ ]:
stopifnot(round(mean(trump_2016$votes),digits=6) == 3709.576408)

Create a chloropleth map that shows the porportion of registered Republican voters that voted for Trump in each county. The variable from the `trump_2016` data set that you should use is `fraction_votes`. Use the `cut` function to create seven interval groups: 0% - 15%, 15% - 30%, 30% - 45%, 45% - 60%, 60% - 75%, 75% - 90%, and above 90%. This will be necessary when creating the color buckets for plotting. Remember to include all of the necessary plot elements like a title, legend, context information, etc.

In [ ]:
# YOUR CODE HERE
install.packages("mapproj")
install.packages("RColorBrewer")
library(RColorBrewer)
colors = brewer.pal(7, "Greens")

library("maps")
data("county.fips")
levels = c(0,0.15,0.3,0.45,0.6,0.75,0.9,1)
trump_2016$colorbuckets = as.numeric(cut(trump_2016$fraction_votes,breaks = levels))
leg.txt = c("0-15%","15-30%","30-45%","45-60%","60-75%","75-90%","above 90%")
map.data = county.fips
plot.data = merge(map.data,trump_2016,by = "fips")

map("county", col = colors[plot.data$colorbuckets], fill = TRUE, resolution = 0, lty = 0, projection = "polyconic")
map("state", col = "white", fill = FALSE, add = TRUE, lty= 1, lwd = 0.2, projection="polyconic")
title("Proportion of registered Republican voters that voted for Trump in each county")
legend("topleft", leg.txt, horiz = TRUE, fill = colors,bty="n")


---

# Problem 2

Next, the `air_qual` data set includes information on the median levels of the pollutant measure PM2.5 at 142 outdoor monitoring stations in California in 2017. Use these data to create an isopleth map of the median PM2.5 levels in 2017. Include all necessary plot elements.

In [ ]:
data("air_qual")
# YOUR CODE HERE

library(maps)
library(AMSdata)
library(RColorBrewer)
library(akima)

x = air_qual
x$logpm2.5 = log(1+x$pm2.5)
map("usa", xlim=c(-124,-116), ylim=c(32, 42), lwd=2, fill=T, col="grey",
     boundary=FALSE, mar=c(1,1,1,1))
colorscale = brewer.pal(8, "PuRd")
scallops.int = interp(x=x$SITE_LONGITUDE, y=x$SITE_LATITUDE, z=x$logpm2.5,
                      xo=seq(min(x$SITE_LONGITUDE), max(x$SITE_LONGITUDE), length = 80),
                      yo=seq(min(x$SITE_LATITUDE), max(x$SITE_LATITUDE), length = 80), linear = F)

image(scallops.int, col=colorscale, axes=F, add=T)
contour(scallops.int, add=T, col="black")
points(x$SITE_LONGITUDE, x$SITE_LATITUDE, cex=0.75, pch=20)



---

# Problem 3

The `lesson_8` data set contains mostly-cleaned text data from the lecture 8 slides. Use this information to produce two plots: a word cloud of the most common words used in the lecture, and a bar plot of the 8 most common words. In order to prevent too many words from appearing in the word cloud, set the minimum number of occurrances for each word to be 5. Include plot elements on the bar plot, but exclude the title, labels, and context information on the word cloud, as including it may be confusing.

In [ ]:
data("lesson_8")
install.packages("tm")
install.packages("SnowballC")
install.packages("wordcloud")
install.packages("RColorBrewer")
# YOUR CODE HERE
library(tm)
library(AMSdata)
library(SnowballC)
library(RColorBrewer)
library(wordcloud)

a = lesson_8
docs = Corpus(VectorSource(a))
docs = tm_map(docs, content_transformer(tolower))
docs = tm_map(docs, removeNumbers)
docs = tm_map(docs, removePunctuation)
docs = tm_map(docs, stripWhitespace)
docs = tm_map(docs, removeWords, stopwords("english"))
dtm = TermDocumentMatrix(docs)
m = as.matrix(dtm)
v = sort(rowSums(m),decreasing=TRUE)
d = data.frame(word = names(v),freq=v)
set.seed(1234)
wordcloud(words = d$word, freq = d$freq, min.freq = 5,max.words=200, random.order=FALSE, rot.per=0.35, colors=brewer.pal(8, "Dark2"))

ord = order(d$freq, decreasing=T)
pal2 = brewer.pal(8,"Dark2")
colscale = c(pal2[8], pal2[7],pal2[5], rep(pal2[4],2),rep(pal2[3],3))
par(mar=c(3,5,1,1)+0.2)
barplot(height=round(d$freq[ord[1:8]]/sum(d$freq), 4),names.arg=NULL, horiz=T, las=1,xlim=c(0,0.03), col=colscale,border=colscale)
mtext(side = 2, line = 1, text =d$word[ord[1:8]], at = seq(0.7,length=8,by=1.2), las=2, col=colscale)

---

# Problem 4

The `new_quakes` data set includes data on earthquakes that registered over a magnitude of 4.5 between October 23, 2017 and November 23, 2017 all over the world. Use this data frame to make a point process map in which the color of each point describes its magnitude. Be sure to include a title, legend, and context information!

In [ ]:
library(AMSdata)
data("new_quakes")
library(ggplot2)
library(ggmap)

# YOUR CODE HERE
x = new_quakes
map.world <- map_data(map = "world")
sp = ggplot(data = x, aes(longitude,latitude))
sp = sp+geom_polygon(data = map.world, aes(long, lat, group = group), colour = "white", size = 0.4)
sp = sp+geom_point(aes(colour = mag, size = mag, fill = mag), alpha = 0.7)
sp = sp+scale_colour_gradient(low = "green", high = "red")

#earthquakes that registered over a magnitude of 4.5 between October 23, 2017 and November 23, 2017
sp + ggtitle("                             Earthquakes that registered over a magnitude of 4.5 between October 23, 2017 and November 23, 2017")